In [132]:
from ebooklib import epub
import ebooklib
book1 = epub.read_epub('./jules_vernes_20000.epub')

/Users/miguelboland/Projects/data_science/env/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/miguelboland/Projects/data_science/env/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [135]:
import unicodedata
from bs4 import BeautifulSoup

all_text = []
for i, doc in enumerate(book1.get_items_of_type(ebooklib.ITEM_DOCUMENT)):
    if i < 20:
        continue
    doc = BeautifulSoup(doc.content.decode(), features="xml")
    ps = doc.find_all('p')
    text = list(filter(bool, [unicodedata.normalize("NFKD", p.get_text()).replace('\n', ' ').strip().replace('— ', '') for p in ps]))
    text = ' '.join(text)
    sentences = [(t + '.') for t in text.split('.')]
    all_text.extend(sentences)
    if i > 200:
        break
for i in range(5):
    print(all_text[i])

À l’époque où ces évènements se produisirent, je revenais d’une exploration scientifique entreprise dans les mauvaises terres du Nébraska, aux États-Unis.
 En ma qualité de professeur suppléant au Muséum d’histoire naturelle de Paris, le gouvernement français m’avait joint à cette expédition.
 Après six mois passés dans le Nébraska, chargé de précieuses collections, j’arrivai à New-York vers la fin de mars.
 Mon départ pour la France était fixé aux premiers jours de mai.
 Je m’occupais donc, en attendant, de classer mes richesses minéralogiques, botaniques et zoologiques, quand arriva l’incident du Scotia.


In [138]:
articles = ['le', 'un', 'une', 'du', 'de la', 'de', 'la', 'son', 'sa', 'des', 'ce', 'cette']
qs = []

def mask_article(article, text):
    text = text.replace(article, ' ___ ', 1)
    text = text.split(' ')
    i = text.index('___')
    
    final_text = ' '.join(text)
    if '.' in final_text:
        full_stop_i = final_text.index('.')
        # if  final_text.index('.') < i:
        #     final_text = final_text[final_text.index('.')+1:]
    return final_text

for text in all_text:
    for article in articles:
        if f' {article.capitalize()} ' in text:
            final_text = mask_article(f' {article.capitalize()} ', text)
        elif f' {article} ' in text:
            final_text = mask_article(f' {article} ', text)
        else:
            continue
            
            
        if len(final_text) < 10 or '___' not in final_text:
            continue
        qs.append([article, final_text])
        if article == 'de la':
            break
qs

[['du',
  'À l’époque où ces évènements se produisirent, je revenais d’une exploration scientifique entreprise dans les mauvaises terres ___ Nébraska, aux États-Unis.'],
 ['le',
  ' En ma qualité de professeur suppléant au Muséum d’histoire naturelle de Paris, ___ gouvernement français m’avait joint à cette expédition.'],
 ['de',
  ' En ma qualité ___ professeur suppléant au Muséum d’histoire naturelle de Paris, le gouvernement français m’avait joint à cette expédition.'],
 ['cette',
  ' En ma qualité de professeur suppléant au Muséum d’histoire naturelle de Paris, le gouvernement français m’avait joint à ___ expédition.'],
 ['le',
  ' Après six mois passés dans ___ Nébraska, chargé de précieuses collections, j’arrivai à New-York vers la fin de mars.'],
 ['de',
  ' Après six mois passés dans le Nébraska, chargé ___ précieuses collections, j’arrivai à New-York vers la fin de mars.'],
 ['la',
  ' Après six mois passés dans le Nébraska, chargé de p

In [139]:
from collections import Counter
print(Counter([x[0] for x in qs]))

Counter({'de': 2869, 'le': 2793, 'la': 1771, 'du': 1222, 'un': 1115, 'une': 1049, 'des': 1022, 'ce': 832, 'de la': 571, 'cette': 541, 'son': 420, 'sa': 355})


In [140]:
import json
problems = {'qs': qs}
with open('./qs.json', 'w') as f:
    json.dump(problems, f)

In [159]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import random

b = widgets.Button(
    description='Show Random',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)
display(b)

out = widgets.Output()
display(out)

def on_button_clicked(b):
    with out:
        clear_output()
        print("New hello from each button click!. This hello from {}.".format(random.choice(list(range(100)))))

b.on_click(on_button_clicked)

Button(button_style='info', description='Show Random', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()